In [3]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [13]:
df = pd.read_csv('./compras.csv')
df.head()


,cliente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,JÚLIO DEMARQUE,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAYCON SOARES,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WEMERSON NETO,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOÃO DEMARQUE,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,MAYCON SANTOS,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
total_de_compras = df.shape[0]

print(total_de_compras)

10000


In [14]:
# Dropando a coluna clientes
df.drop(['cliente'], axis=1, inplace=True)

In [18]:
#Verificando os produtos
uniques = df['0'].unique()
print(uniques)

['limão' 'salsicha' 'queijo' 'pimenta' 'detergente' 'maionese'
 'molho branco' 'azeitona' 'vinho tinto' 'mostarda' 'picanha' 'manga'
 'achocolatado' 'banana' 'sopa pronta' 'refrigerante' 'óleo de soja'
 'bolo' 'kiwi' 'laranja' 'mel' 'atum enlatado' 'cereal' 'molho pesto'
 'massa de pizza' 'energético' 'torrada' 'feijão' 'vinagre' 'whisky'
 'morango' 'batata' 'leite' 'manteiga' 'linguiça' 'papel higiênico'
 'verduras' 'chocolate' 'café' 'azeite' 'chá' 'espumante' 'arroz'
 'sardinha enlatada' 'açúcar' 'suco' 'milho verde' 'pepino' 'uva' 'pão'
 'lasanha' 'vodka' 'amendoim' 'cebola' 'bisnaguinha' 'gelatina' 'frango'
 'sorvete' 'nhoque' 'geleia' 'filé de frango' 'patê' 'carvão' 'frutas'
 'presunto' 'palmito' 'molho de tomate' 'batata frita' 'pudim' 'nutella'
 'maçã' 'barra de cereal' 'farinha de trigo' 'peito de peru'
 'cerveja artesanal' 'água tônica' 'carne moída' 'alface'
 'suco de laranja' 'sabão' 'macarrão' 'vinho' 'catchup' 'cenoura'
 'vinho branco' 'sal' 'ervilha' 'granola' 'biscoito

In [22]:
#renomiando algumas colunas
replaces = {
    'alface': 'salada',
    'vinho tinto': 'vinho',
    'vinho branco': 'vinho',
    'salada':'verduras'
}

df.replace(replaces, inplace=True)

In [28]:
#Mais vendidos

count = df.stack().value_counts()
count.head(20)

verduras             1853
queijo               1241
vinho                1178
presunto             1129
pão                  1056
feijão               1040
picanha               869
arroz                 856
manteiga              803
cerveja               759
leite                 722
bolo                  716
salsicha              682
açúcar                631
sorvete               621
carvão                612
batata frita          607
cerveja artesanal     581
tomate                577
refrigerante          573
Name: count, dtype: int64

In [29]:
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])

regras = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)

resultados = list(regras)
resultados

[RelationRecord(items=frozenset({'feijão', 'arroz'}), support=0.0544, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'feijão'}), confidence=0.6384976525821596, lift=6.223174001775435), OrderedStatistic(items_base=frozenset({'feijão'}), items_add=frozenset({'arroz'}), confidence=0.530214424951267, lift=6.223174001775435)]),
 RelationRecord(items=frozenset({'filé de frango', 'arroz'}), support=0.0249, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'filé de frango'}), confidence=0.29225352112676056, lift=7.731574632983084), OrderedStatistic(items_base=frozenset({'filé de frango'}), items_add=frozenset({'arroz'}), confidence=0.6587301587301587, lift=7.731574632983084)]),
 RelationRecord(items=frozenset({'frango', 'arroz'}), support=0.0425, ordered_statistics=[OrderedStatistic(items_base=frozenset({'arroz'}), items_add=frozenset({'frango'}), confidence=0.49882629107981225, lift=8.813185354767), OrderedSt

In [50]:
resultados_lista = []

# Loop sobre os resultados e preencher a lista
for resultado in resultados:
    items_base = list(resultado.items)[0]
    items_add = list(resultado.items)[1]
    regra = f"{items_base} => {items_add}"
    suporte = resultado.support
    confianca = resultado.ordered_statistics[0].confidence
    lift = resultado.ordered_statistics[0].lift
    
    # Adicionando o dicionário à lista
    resultados_lista.append({'Regras': regra,
                             'Suporte': suporte,
                             'Lift': lift,
                             'Confiança': confianca,
                             'Confiança (%)': confianca * 100
                             })

# Criando o DataFrame a partir da lista de dicionários
resultados_df = pd.DataFrame(resultados_lista)
resultados_df = resultados_df.sort_values(by='Lift',ascending=False)
# Exibindo o DataFrame
resultados_df.head()




,Regras,Suporte,Lift,Confiança,Confiança (%)
40,pizza => refrigerante,0.0243,11.058423,0.621483,62.148338
12,hambúrguer => batata frita,0.0229,10.967433,0.381667,38.166667
8,açúcar => café,0.0238,10.198135,0.381410,38.141026
31,sorvete => chocolate,0.0241,10.145147,0.614796,61.479592
28,pizza => cerveja artesanal,0.0201,9.082447,0.355124,35.512367


In [31]:
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

# Converte o array resultante em um DataFrame pandas
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Agora você pode aplicar o fpgrowth no DataFrame one-hot
frequent_itemsets = fpgrowth(df_onehot, min_support=0.003,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

rules.size

13940

In [51]:
# imprimindo as regras mais importantes ordenadas por lift
rules = rules.sort_values(by='lift', ascending=False)
rules.head(-1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
859,"(mostarda, batata frita, cerveja)","(hambúrguer, catchup)",0.0056,0.0145,0.0041,0.732143,50.492611,0.004019,3.679200,0.985715
870,"(hambúrguer, catchup)","(mostarda, batata frita, cerveja)",0.0145,0.0056,0.0041,0.282759,50.492611,0.004019,1.386423,0.994617
865,"(mostarda, hambúrguer)","(batata frita, catchup, cerveja)",0.0110,0.0074,0.0041,0.372727,50.368550,0.004019,1.582406,0.991048
864,"(batata frita, catchup, cerveja)","(mostarda, hambúrguer)",0.0074,0.0110,0.0041,0.554054,50.368550,0.004019,2.217758,0.987453
816,"(hambúrguer, catchup)","(mostarda, batata frita)",0.0145,0.0092,0.0063,0.434483,47.226387,0.006167,1.752024,0.993227
...,...,...,...,...,...,...,...,...,...,...
1039,(leite),(açúcar),0.0707,0.0624,0.0138,0.195191,3.128060,0.009388,1.164997,0.732070
1038,(açúcar),(leite),0.0624,0.0707,0.0138,0.221154,3.128060,0.009388,1.193175,0.725590
736,(batata frita),(cerveja),0.0600,0.0754,0.0141,0.235000,3.116711,0.009576,1.208627,0.722499
737,(cerveja),(batata frita),0.0754,0.0600,0.0141,0.187003,3.116711,0.009576,1.156215,0.734533
